In [16]:
import pandas as pd
import numpy as np
import json
import os
import requests
from bs4 import BeautifulSoup

In [17]:
url = "https://www.bball-index.com/lebron-database/"
response = requests.get(url)

response.status_code

200

In [18]:
soup = BeautifulSoup(response.text, "html.parser")

In [19]:
#Get LEBRON metrics 

table = soup.select_one("div#content.site-content table#table_1")

if table:
    # Extract column headers
    headers = [header.text.strip() for header in table.find_all("th")]

    # Initialize an empty list to store the rows of data
    data_rows = []

    # Extract data from the table
    for row in table.find_all("tr")[1:]:  # Skip the first row as it contains headers
        cells = row.find_all(["td", "th"])
        if cells:
            # Append the row data to the list
            data_rows.append([cell.text.strip() for cell in cells])

    # Create a DataFrame
    lebron_df = pd.DataFrame(data_rows, columns=headers)
    
lebron_df.head()

,bref_id,Player,Season,Team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON
0,hillgr01,Grant Hill,2009-10,PHO,Athletic Finisher,"2,430",81,SF,37,-0.46,-0.82,0.36,3.00,"5,268,027",255,0.66,0.52,0.14
1,hunteli01,Lindsey Hunter,2009-10,CHI,Secondary Ball Handler,122,13,PG,39,-1.00,-0.24,-0.76,0.10,"981,535",283,-0.69,-0.35,-0.34
2,onealsh01,Shaquille O'Neal,2009-10,CLE,Post Scorer,"1,240",53,C,37,0.29,-1.37,1.65,2.20,"4,034,036",406,0.97,-0.28,1.25
3,howarju01,Juwan Howard,2009-10,POR,Post Scorer,"1,632",73,C,36,-3.49,-2.19,-1.29,-1.30,"-1,118,330",436,-2.43,-2.01,-0.42
4,kiddja01,Jason Kidd,2009-10,DAL,Stationary Shooter,"2,881",80,PG,36,2.87,1.24,1.63,10.00,"15,762,060",467,3.27,1.81,1.46


In [20]:
#Clean Data

lebron_df = lebron_df.drop("bref_id", axis = 1)

lebron_df.head()

,Player,Season,Team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON
0,Grant Hill,2009-10,PHO,Athletic Finisher,"2,430",81,SF,37,-0.46,-0.82,0.36,3.00,"5,268,027",255,0.66,0.52,0.14
1,Lindsey Hunter,2009-10,CHI,Secondary Ball Handler,122,13,PG,39,-1.00,-0.24,-0.76,0.10,"981,535",283,-0.69,-0.35,-0.34
2,Shaquille O'Neal,2009-10,CLE,Post Scorer,"1,240",53,C,37,0.29,-1.37,1.65,2.20,"4,034,036",406,0.97,-0.28,1.25
3,Juwan Howard,2009-10,POR,Post Scorer,"1,632",73,C,36,-3.49,-2.19,-1.29,-1.30,"-1,118,330",436,-2.43,-2.01,-0.42
4,Jason Kidd,2009-10,DAL,Stationary Shooter,"2,881",80,PG,36,2.87,1.24,1.63,10.00,"15,762,060",467,3.27,1.81,1.46
